<h1 style="color:blue;"> Feature Extraction and Data Preparation</h1>

<div class="span5 alert alert-info">
    Applying Feature Extraction to the given dataset and preparaing data for Model Building.<br><br>
    
    <ul>    **Explore Below Features individually and in Combination **
        <li> Category
        <li>Company
        <li>Brand
        <li> Purchase Amount
        <li>Purchase Quantity
        <li>Offer
        <li>Offerdate
        <li>Transaction date
    </ul>
    
    <p>Finally Save the dataframe to a CSV file for Machine Learning</p>
</div>

In [1]:
import pandas as pd
import numpy as np

<div class="span5 alert alert-info">Download data for this project into data folder (extract the zip file) <br>
    [transactions](https://drive.google.com/file/d/1GQDqcLaRgLA7WpPkhTjD6gJ7MsgJDXJJ/view?usp=sharing)<br>
    [history](https://drive.google.com/file/d/1bQEaPp5WWUzMn19SgdpBkx-0FrxMKrss/view?usp=sharing)<br>
    [offers](https://drive.google.com/file/d/187pRzBYddjrSNlInz2R6liHKaQ99j-R-/view?usp=sharing)<br>
    </div>

<h1 style="color:blue;"> Loading data</h1>

In [2]:

# Cust Offer History
hist = pd.read_csv("../../data/trans_Hist16000.csv")

# Offer Information
offers = pd.read_csv("../../data/offers.csv")

# Transactions for customer who received Offer
trans = pd.read_csv("../../data/transactions_16000.csv")

<h1 style="color:blue;"> Merge History and Offer data on offerId</h1>

### Replacing true/false with 1/0

In [3]:
hist_offer = hist.merge(offers,left_on='offer',right_on='offer')
replace_repeater = {"repeater":     {"t": 1, "f": 0}}
hist_offer.replace(replace_repeater,inplace=True)
hist_offer.head()

,id,chain,offer,market,repeattrips,repeater,offerdate,category,quantity,company,offervalue,brand
0,705936120,165,1208329,21,0,0,2013-04-23,2119,1,108079383,1.0,6926
1,3814736505,77,1208329,37,2,1,2013-04-22,2119,1,108079383,1.0,6926
2,4627974746,153,1208329,12,10,1,2013-04-23,2119,1,108079383,1.0,6926
3,4623268450,153,1208329,12,0,0,2013-04-25,2119,1,108079383,1.0,6926
4,501138510,21,1208329,10,0,0,2013-04-22,2119,1,108079383,1.0,6926


<h1 style="color:blue;">Aggregating Transaction count , purchaseamount and quantity for Customer</h1>

In [31]:
trans['total_transactions_count'] = ''
trans['total_purchase_category_count']=''
trans['total_purchase_company_count']=''
trans['total_purchase_brand_count']=''
trans['time_period']=''

# Transactions Aggregation by ID
agg_trans = pd.DataFrame(trans.groupby(['id']).agg({'total_transactions_count':"count",'purchaseamount':sum,'purchasequantity':sum})).reset_index()
agg_trans.columns = ['id','total_purchaseamount','total_purchasequantity','total_transactions_count']
agg_trans.head()

,id,total_purchaseamount,total_purchasequantity,total_transactions_count
0,13089312,3890.68,1469,1218
1,13873775,6088.08,2955,1463
2,16138642,1872.07,734,537
3,16238376,1836.97,722,576
4,17486592,8078.55,2014,1526


<h1 style="color:blue;"> Transactions Aggregated for Customer and Category</h1>

In [32]:
agg_trans_cat = pd.DataFrame(trans.groupby(['id','category']).agg({'total_purchase_category_count':"count"})).reset_index().fillna(0)
agg_trans_cat.head()

,id,category,total_purchase_category_count
0,13089312,0,5
1,13089312,202,1
2,13089312,214,1
3,13089312,302,7
4,13089312,303,1


<h1 style="color:blue;"> Transactions Aggregated for Customer and Company</h1>

In [33]:
agg_trans_comp = pd.DataFrame(trans.groupby(['id','company']).agg({'total_purchase_company_count':"count"})).reset_index().fillna(0)
agg_trans_comp.head()

,id,company,total_purchase_company_count
0,13089312,10000,16
1,13089312,10980090,7
2,13089312,101087818,1
3,13089312,101090010,1
4,13089312,101200010,1


<h1 style="color:blue;"> Transactions Aggregated for Customer and Brand</h1>

In [34]:
agg_trans_brand = pd.DataFrame(trans.groupby(['id','brand']).agg({'total_purchase_brand_count':"count"})).reset_index().fillna(0)
agg_trans_brand.head()

,id,brand,total_purchase_brand_count
0,13089312,0,42
1,13089312,713,1
2,13089312,885,1
3,13089312,915,28
4,13089312,1008,1


<h1 style="color:blue;"> Merge HIST_OFFER with Transaction Aggregations over Category, Company and Brand</h1>

In [35]:
k1 = pd.merge(hist_offer[['id','offer','repeater','repeattrips','company','category','brand']], agg_trans_comp,on=('id','company'),how='left').fillna(0)
k2 = pd.merge(k1, agg_trans_cat,on=('id','category'),how='left').fillna(0)
k3 = pd.merge(k2, agg_trans_brand,on=('id','brand'),how='left').fillna(0)
k4 = pd.merge(agg_trans,k3 ,on=('id'),how='left').fillna(0)
k4.fillna(0,inplace=True)
k4.head()

,id,total_purchaseamount,total_purchasequantity,total_transactions_count,offer,repeater,repeattrips,company,category,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count
0,13089312,3890.68,1469,1218,1204821,0,0,107717272,5619,102504,3.0,0.0,3.0
1,13873775,6088.08,2955,1463,1197502,0,0,106414464,3203,13474,16.0,0.0,0.0
2,16138642,1872.07,734,537,1204576,0,0,104610040,5616,15889,2.0,0.0,2.0
3,16238376,1836.97,722,576,1197502,1,1,106414464,3203,13474,0.0,5.0,0.0
4,17486592,8078.55,2014,1526,1197502,1,1,106414464,3203,13474,0.0,1.0,0.0


In [36]:
k4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16000 entries, 0 to 15999
Data columns (total 13 columns):
id                               16000 non-null int64
total_purchaseamount             16000 non-null float64
total_purchasequantity           16000 non-null int64
total_transactions_count         16000 non-null int64
offer                            16000 non-null int64
repeater                         16000 non-null int64
repeattrips                      16000 non-null int64
company                          16000 non-null int64
category                         16000 non-null int64
brand                            16000 non-null int64
total_purchase_company_count     16000 non-null float64
total_purchase_category_count    16000 non-null float64
total_purchase_brand_count       16000 non-null float64
dtypes: float64(4), int64(9)
memory usage: 1.7 MB


<h1 style="color:blue;"> Date Based Aggregations from Offerdate</h1>

### Calculate time period between Offerdate and transaction date for all transactions of Customers

In [104]:
# Offer Date Based Transaction Aggregation
trans = pd.merge(hist_offer[['id','offerdate']], trans,on=('id')).fillna(0)
trans['hr']=0
#trans['cnt']=1

trans['date'] = pd.to_datetime(trans['date'])
trans['date'] += pd.to_timedelta(trans['hr'], unit='h')

trans['offerdate'] = pd.to_datetime(trans['offerdate'])
trans['offerdate'] += pd.to_timedelta(trans['hr'], unit='h')

trans.index = trans['date']

trans['time_period'] = ((trans.offerdate - trans.date)/np.timedelta64(1, 'M'))

trans['time_period'] = trans['time_period'].astype(int)


<h1 style="color:blue;"> Aggregating Transactions for Customer over Category and Time Period </h1>

In [ ]:
trans_cat_sum=trans.groupby(['id','category','time_period']).sum().reset_index()

# remove the redundant columns
del trans_cat_sum['index']
del trans_cat_sum['chain']
del trans_cat_sum['company']
del trans_cat_sum['dept']
del trans_cat_sum['brand']
del trans_cat_sum['productsize']
del trans_cat_sum['hr']

trans_cat_sum.head(10)

<h1 style="color:blue;">Aggregate Transactions on 1 Month, 3 Months, 9 Months and 12 Months</h1>

In [68]:
trans_mntly_detls = trans_cat_sum[trans_cat_sum['time_period']<1]
trans_monthly_summary = trans_mntly_detls.groupby(['id']).sum().reset_index()
trans_qtly_detls = trans_cat_sum[trans_cat_sum['time_period']<3]
trans_qtly_summary = trans_qtly_detls.groupby(['id']).sum().reset_index()
trans_half_yr_detls = trans_cat_sum[trans_cat_sum['time_period']<6]
trans_half_yr_summary = trans_half_yr_detls.groupby(['id']).sum().reset_index()
trans_9_mnts_detls = trans_cat_sum[trans_cat_sum['time_period']<9]
trans_9_mnts_summary = trans_9_mnts_detls.groupby(['id']).sum().reset_index()
trans_12_mnts_detls = trans_cat_sum[trans_cat_sum['time_period']<12]
trans_12_mnts_summary = trans_12_mnts_detls.groupby(['id']).sum().reset_index()

trans_monthly_summary = trans_monthly_summary[['id','purchaseamount']]
trans_qtly_summary = trans_qtly_summary[['id','purchaseamount']]
trans_half_yr_summary = trans_half_yr_summary[['id','purchaseamount']]
trans_9_mnts_summary = trans_9_mnts_summary[['id','purchaseamount']]
trans_12_mnts_summary = trans_12_mnts_summary[['id','purchaseamount']]

trans_monthly_summary.columns = ['id','1_month_total_purchase_amt']
trans_qtly_summary.columns = ['id','3_mnths_total_purchase_amt']
trans_half_yr_summary.columns = ['id','6_mnths_total_purchase_amt']
trans_9_mnts_summary.columns = ['id','9_mnths_total_purchase_amt']
trans_12_mnts_summary.columns = ['id','12_mnths_total_purchase_amt']

<h1 style="color:blue;"> Joining  Aggregated Features from above</h1>

In [69]:
k5 = pd.merge(trans_monthly_summary, k4,on=('id'),how='right').fillna(0)
k6 = pd.merge(trans_qtly_summary, k5,on=('id'),how='right').fillna(0)
k7 = pd.merge(trans_half_yr_summary, k6,on=('id'),how='right').fillna(0)
k8 = pd.merge(trans_9_mnts_summary, k7,on=('id'),how='right').fillna(0)
k9 = pd.merge(trans_12_mnts_summary, k8,on=('id'),how='right').fillna(0)
k9.head()

,id,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,offer,repeater,repeattrips,company,category,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count
0,13089312,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,1204821,0,0,107717272,5619,102504,3.0,0.0,3.0
1,13873775,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,1197502,0,0,106414464,3203,13474,16.0,0.0,0.0
2,16138642,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,1204576,0,0,104610040,5616,15889,2.0,0.0,2.0
3,16238376,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,1197502,1,1,106414464,3203,13474,0.0,5.0,0.0
4,17486592,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,1197502,1,1,106414464,3203,13474,0.0,1.0,0.0


<h1 style="color:blue;"> Add a feature for average purchase amount for all transactions</h1>

In [70]:
trans_summary_avg = trans_cat_sum.groupby(['id']).mean().reset_index()
trans_summary_avg = trans_summary_avg[['id','purchaseamount']]
trans_summary_avg.columns = ['id','total_trans_purchaseamount_avg']
k10 = pd.merge(trans_summary_avg, k9,on=('id'),how='right').fillna(0)
k10.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,offer,repeater,repeattrips,company,category,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,1204821,0,0,107717272,5619,102504,3.0,0.0,3.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,1197502,0,0,106414464,3203,13474,16.0,0.0,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,1204576,0,0,104610040,5616,15889,2.0,0.0,2.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,1197502,1,1,106414464,3203,13474,0.0,5.0,0.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,1197502,1,1,106414464,3203,13474,0.0,1.0,0.0


<h1 style="color:blue;"> Average purchase amount for offered category</h1>

In [71]:
trans_summary_avg_cat = trans_cat_sum.groupby(['id','category']).mean().reset_index()
trans_summary_avg_cat = trans_summary_avg_cat[['id','category','purchaseamount']]
trans_summary_avg_cat.columns = ['id','category','purchaseamount_category_avg']
k11 = pd.merge(k10, trans_summary_avg_cat,on=(['id','category']),how='left').fillna(0)
k11.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,offer,repeater,repeattrips,company,category,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count,purchaseamount_category_avg
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,1204821,0,0,107717272,5619,102504,3.0,0.0,3.0,0.00
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,1197502,0,0,106414464,3203,13474,16.0,0.0,0.0,0.00
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,1204576,0,0,104610040,5616,15889,2.0,0.0,2.0,0.00
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,1197502,1,1,106414464,3203,13474,0.0,5.0,0.0,3.41
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,1197502,1,1,106414464,3203,13474,0.0,1.0,0.0,3.49


<h1 style="color:blue;"> Extract Month in which Offer was given to customer</h1>

In [72]:
hist_offer['hr']=0
hist_offer['offerdate'] = pd.to_datetime(hist_offer['offerdate'])
hist_offer['offerdate'] += pd.to_timedelta(hist_offer['hr'], unit='h')
hist_offer['offeredmonth'] = pd.DatetimeIndex(hist_offer['offerdate']).month

k12 = pd.merge(k11, hist_offer[['id','offervalue','offeredmonth','quantity']],on=(['id']),how='left').fillna(0)
k12.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,...,company,category,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count,purchaseamount_category_avg,offervalue,offeredmonth,quantity
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,107717272,5619,102504,3.0,0.0,3.0,0.00,1.50,4,1
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,106414464,3203,13474,16.0,0.0,0.0,0.00,0.75,3,1
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,104610040,5616,15889,2.0,0.0,2.0,0.00,1.00,4,1
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,106414464,3203,13474,0.0,5.0,0.0,3.41,0.75,4,1
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,106414464,3203,13474,0.0,1.0,0.0,3.49,0.75,4,1


<h1 style="color:blue;"> Purchase Quantity and Spending on Offered Category in last 30 days</h1>

In [73]:
#Category
trans_mntly_detls.loc[:,'category_purchased_amt_30days']=1
trans_mntly_detls.loc[:,'category_purchased_qty_30days']=1
p = trans_mntly_detls.groupby(['id','category']).sum().reset_index()

k13 = pd.merge(k12, p[['id','category','purchaseamount','purchasequantity']],on=(['id','category']),how='left').fillna(0)
k13.rename(index=str, columns={'purchaseamount':'category_purchased_amt_30days','purchasequantity':'category_purchased_qty_30days'},inplace=True)
k13.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,...,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count,purchaseamount_category_avg,offervalue,offeredmonth,quantity,category_purchased_amt_30days,category_purchased_qty_30days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,102504,3.0,0.0,3.0,0.00,1.50,4,1,0.0,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,13474,16.0,0.0,0.0,0.00,0.75,3,1,0.0,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,15889,2.0,0.0,2.0,0.00,1.00,4,1,0.0,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,13474,0.0,5.0,0.0,3.41,0.75,4,1,0.0,0.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,13474,0.0,1.0,0.0,3.49,0.75,4,1,0.0,0.0


<h1 style="color:blue;"> Purchase Quantity and Spending on Offered Category in last 90 days</h1>

In [74]:
trans_qtly_detls.loc[:,'category_purchased_amt_90days']=1
trans_qtly_detls.loc[:,'category_purchased_qty_90days']=1
p1 = trans_qtly_detls.groupby(['id','category']).sum().reset_index()
k14 = pd.merge(k13, p1[['id','category','purchaseamount','purchasequantity']],on=(['id','category']),how='left').fillna(0)
#k14.columns = ['id','category','category_purchased_90days']
k14.rename(index=str, columns={'purchaseamount':'category_purchased_amt_90days','purchasequantity':'category_purchased_qty_90days'},inplace=True)
k14.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,...,total_purchase_category_count,total_purchase_brand_count,purchaseamount_category_avg,offervalue,offeredmonth,quantity,category_purchased_amt_30days,category_purchased_qty_30days,category_purchased_amt_90days,category_purchased_qty_90days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,0.0,3.0,0.00,1.50,4,1,0.0,0.0,0.00,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,0.0,0.0,0.00,0.75,3,1,0.0,0.0,0.00,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,0.0,2.0,0.00,1.00,4,1,0.0,0.0,0.00,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,5.0,0.0,3.41,0.75,4,1,0.0,0.0,4.48,2.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,1.0,0.0,3.49,0.75,4,1,0.0,0.0,0.00,0.0


<h1 style="color:blue;"> Purchase Quantity and Spending on Offered Category in last 180 days</h1>

In [75]:
trans_half_yr_detls.loc[:,'category_purchased_amt_180days']=1
trans_half_yr_detls.loc[:,'category_purchased_qty_180days']=1
p2 = trans_half_yr_detls.groupby(['id','category']).sum().reset_index()
k15 = pd.merge(k14, p2[['id','category','purchaseamount','purchasequantity']],on=(['id','category']),how='left').fillna(0)
#k15.columns = ['id','category','category_purchased_180days']
k15.rename(index=str, columns={'purchaseamount':'category_purchased_amt_180days','purchasequantity':'category_purchased_qty_180days'},inplace=True)
k15.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,...,purchaseamount_category_avg,offervalue,offeredmonth,quantity,category_purchased_amt_30days,category_purchased_qty_30days,category_purchased_amt_90days,category_purchased_qty_90days,category_purchased_amt_180days,category_purchased_qty_180days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,0.00,1.50,4,1,0.0,0.0,0.00,0.0,0.00,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,0.00,0.75,3,1,0.0,0.0,0.00,0.0,0.00,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,0.00,1.00,4,1,0.0,0.0,0.00,0.0,0.00,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,3.41,0.75,4,1,0.0,0.0,4.48,2.0,4.48,2.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,3.49,0.75,4,1,0.0,0.0,0.00,0.0,0.00,0.0


<h1 style="color:blue;"> Purchase Quantity and Spending on Offered Category in last 270 days</h1>

In [76]:
trans_9_mnts_detls.loc[:,'category_purchased_amt_270days']=1
trans_9_mnts_detls.loc[:,'category_purchased_qty_270days']=1
p3 = trans_9_mnts_detls.groupby(['id','category']).sum().reset_index()
k16 = pd.merge(k15, p3[['id','category','purchaseamount','purchasequantity']],on=(['id','category']),how='left').fillna(0)
#k16.columns = ['id','category','category_purchased_270days']
k16.rename(index=str, columns={'purchaseamount':'category_purchased_amt_270days','purchasequantity':'category_purchased_qty_270days'},inplace=True)
k16.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,...,offeredmonth,quantity,category_purchased_amt_30days,category_purchased_qty_30days,category_purchased_amt_90days,category_purchased_qty_90days,category_purchased_amt_180days,category_purchased_qty_180days,category_purchased_amt_270days,category_purchased_qty_270days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,4,1,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,3,1,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,4,1,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,4,1,0.0,0.0,4.48,2.0,4.48,2.0,4.48,2.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,4,1,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0


<h1 style="color:blue;"> Aggregating Transactions for Customer over Company and Time Period </h1>

In [113]:
# Company
trans_comp_sum=trans.groupby(['id','company','time_period']).sum().reset_index()

del trans_comp_sum['index']
del trans_comp_sum['chain']
del trans_comp_sum['category']
del trans_comp_sum['dept']
del trans_comp_sum['brand']
del trans_comp_sum['productsize']
del trans_comp_sum['hr']

###
trans_mntly_detls = trans_comp_sum[trans_comp_sum['time_period']<1]
trans_monthly_summary = trans_mntly_detls.groupby(['id']).sum().reset_index()
trans_qtly_detls = trans_comp_sum[trans_comp_sum['time_period']<3]
trans_qtly_summary = trans_qtly_detls.groupby(['id']).sum().reset_index()
trans_half_yr_detls = trans_comp_sum[trans_comp_sum['time_period']<6]
trans_half_yr_summary = trans_half_yr_detls.groupby(['id']).sum().reset_index()
trans_9_mnts_detls = trans_comp_sum[trans_comp_sum['time_period']<9]
trans_9_mnts_summary = trans_9_mnts_detls.groupby(['id']).sum().reset_index()
trans_12_mnts_detls = trans_comp_sum[trans_comp_sum['time_period']<12]
trans_12_mnts_summary = trans_12_mnts_detls.groupby(['id']).sum().reset_index()

<h1 style="color:blue;"> Purchase Quantity and Spending on Offered Company in last 30, 90, 180 270 days</h1>

In [116]:
###
trans_mntly_detls['company_purchased_amt_30days']=1
trans_mntly_detls['company_purchased_qty_30days']=1

p4 = trans_mntly_detls.groupby(['id','company']).sum().reset_index()
k17 = pd.merge(k16, p4[['id','company','purchaseamount','purchasequantity']],on=(['id','company']),how='left').fillna(0)
k17.rename(index=str, columns={'purchaseamount':'company_purchased_amt_30days','purchasequantity':'company_purchased_qty_30days'},inplace=True)
k17.head()

trans_qtly_detls['company_purchased_amt_90days']=1
trans_qtly_detls['company_purchased_qty_90days']=1

p5 = trans_qtly_detls.groupby(['id','company']).sum().reset_index()
k18 = pd.merge(k17, p5[['id','company','purchaseamount','purchasequantity']],on=(['id','company']),how='left').fillna(0)
#k18.columns = ['id','company','company_purchased_90days']
k18.rename(index=str, columns={'purchaseamount':'company_purchased_amt_90days','purchasequantity':'company_purchased_qty_90days'},inplace=True)

trans_half_yr_detls['company_purchased_amt_180days']=1
trans_half_yr_detls['company_purchased_qty_180days']=1

p6 = trans_half_yr_detls.groupby(['id','company']).sum().reset_index()
k19 = pd.merge(k18, p6[['id','company','purchaseamount','purchasequantity']],on=(['id','company']),how='left').fillna(0)
#k19.columns = ['id','company','company_purchased_180days']
k19.rename(index=str, columns={'purchaseamount':'company_purchased_amt_180days','purchasequantity':'company_purchased_qty_180days'},inplace=True)

trans_9_mnts_detls['company_purchased_amt_270days']=1
trans_9_mnts_detls['company_purchased_qty_270days']=1

p7 = trans_9_mnts_detls.groupby(['id','company']).sum().reset_index()
k20 = pd.merge(k19, p7[['id','company','purchaseamount','purchasequantity']],on=(['id','company']),how='left').fillna(0)
#k20.columns = ['id','company','company_purchased_270days']
k20.rename(index=str, columns={'purchaseamount':'company_purchased_amt_270days','purchasequantity':'company_purchased_qty_270days'},inplace=True)
k20.head()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,...,category_purchased_amt_270days,category_purchased_qty_270days,company_purchased_amt_30days,company_purchased_qty_30days,company_purchased_amt_90days,company_purchased_qty_90days,company_purchased_amt_180days,company_purchased_qty_180days,company_purchased_amt_270days,company_purchased_qty_270days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,0.00,0.0,7.98,2.0,19.95,5.0,19.95,5.0,19.95,5.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,0.00,0.0,4.70,5.0,11.18,15.0,38.00,33.0,53.43,48.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,0.00,0.0,0.00,0.0,3.99,1.0,8.98,2.0,8.98,2.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,4.48,2.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0


<h1 style="color:blue;"> Aggregating Transactions for Customer over Brand and Time Period </h1>

In [117]:
#Brand
trans['cnt']=1
trans_brand_sum=trans.groupby(['id','brand','time_period']).sum().reset_index()

del trans_brand_sum['index']
del trans_brand_sum['chain']
del trans_brand_sum['company']
del trans_brand_sum['dept']
del trans_brand_sum['category']
del trans_brand_sum['productsize']
del trans_brand_sum['hr']


<h1 style="color:blue;"> Purchase Quantity and Spending on Offered Brand in last 30, 90, 180 270 days</h1>

In [118]:
###
trans_mntly_detls = trans_brand_sum[trans_brand_sum['time_period']<1]
trans_monthly_summary = trans_mntly_detls.groupby(['id']).sum().reset_index()
trans_qtly_detls = trans_brand_sum[trans_brand_sum['time_period']<3]
trans_qtly_summary = trans_qtly_detls.groupby(['id']).sum().reset_index()
trans_half_yr_detls = trans_brand_sum[trans_brand_sum['time_period']<6]
trans_half_yr_summary = trans_half_yr_detls.groupby(['id']).sum().reset_index()
trans_9_mnts_detls = trans_brand_sum[trans_brand_sum['time_period']<9]
trans_9_mnts_summary = trans_9_mnts_detls.groupby(['id']).sum().reset_index()
trans_12_mnts_detls = trans_brand_sum[trans_brand_sum['time_period']<12]
trans_12_mnts_summary = trans_12_mnts_detls.groupby(['id']).sum().reset_index()

###
trans_mntly_detls['brand_purchased_amt_30days']=1
trans_mntly_detls['brand_purchased_qty_30days']=1

p5 = trans_mntly_detls.groupby(['id','brand']).sum().reset_index()
k21 = pd.merge(k20, p5[['id','brand','purchaseamount','purchasequantity']],on=(['id','brand']),how='left').fillna(0)
k21.rename(index=str, columns={'purchaseamount':'brand_purchased_amt_30days','purchasequantity':'brand_purchased_qty_30days'},inplace=True)

trans_qtly_detls['brand_purchased_amt_90days']=1
trans_qtly_detls['brand_purchased_qty_90days']=1

p6 = trans_qtly_detls.groupby(['id','brand']).sum().reset_index()
k22 = pd.merge(k21, p6[['id','brand','purchaseamount','purchasequantity']],on=(['id','brand']),how='left').fillna(0)
k22.rename(index=str, columns={'purchaseamount':'brand_purchased_amt_90days','purchasequantity':'brand_purchased_qty_90days'},inplace=True)

trans_half_yr_detls['brand_purchased_amt_180days']=1
trans_half_yr_detls['brand_purchased_qty_180days']=1

p7 = trans_half_yr_detls.groupby(['id','brand']).sum().reset_index()
k23 = pd.merge(k22, p7[['id','brand','purchaseamount','purchasequantity']],on=(['id','brand']),how='left').fillna(0)
k23.rename(index=str, columns={'purchaseamount':'brand_purchased_amt_180days','purchasequantity':'brand_purchased_qty_180days'},inplace=True)

trans_9_mnts_detls['brand_purchased_amt_270days']=1
trans_9_mnts_detls['brand_purchased_qty_270days']=1

p8 = trans_9_mnts_detls.groupby(['id','brand']).sum().reset_index()
k24 = pd.merge(k23, p8[['id','brand','purchaseamount','purchasequantity']],on=(['id','brand']),how='left').fillna(0)
k24.rename(index=str, columns={'purchaseamount':'brand_purchased_amt_270days','purchasequantity':'brand_purchased_qty_270days'},inplace=True)
k24.head()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,...,company_purchased_amt_270days,company_purchased_qty_270days,brand_purchased_amt_30days,brand_purchased_qty_30days,brand_purchased_amt_90days,brand_purchased_qty_90days,brand_purchased_amt_180days,brand_purchased_qty_180days,brand_purchased_amt_270days,brand_purchased_qty_270days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,19.95,5.0,7.98,2.0,19.95,5.0,19.95,5.0,19.95,5.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,53.43,48.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,8.98,2.0,0.00,0.0,3.99,1.0,8.98,2.0,8.98,2.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0


<h1 style="color:blue;"> Aggregating number of purchases by Customer for offered Category, Company and Brand</h1>

In [119]:
trans['purchased_company_category_brand_count']=1
trans_cat_comp_brnd_sum=trans.groupby(['id','company','category','brand']).count().reset_index()
trans_cat_comp_brnd_sum = trans_cat_comp_brnd_sum[['id','company','category','brand','purchased_company_category_brand_count']]

k25 = pd.merge(k24, trans_cat_comp_brnd_sum,on=(['id','company','category','brand']),how='left').fillna(0)

trans['purchased_company_category_count']=1
trans_cat_comp_sum=trans.groupby(['id','company','category']).sum().reset_index()
trans_cat_comp_sum = trans_cat_comp_sum[['id','company','category','purchased_company_category_count']]

k26 = pd.merge(k25, trans_cat_comp_sum,on=(['id','company','category']),how='left').fillna(0)
k26.rename(index=str, columns={'company':'offer_company','category':'offer_category','brand':'offer_brand'},inplace=True)
k26.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,total_purchaseamount,total_purchasequantity,total_transactions_count,...,brand_purchased_amt_30days,brand_purchased_qty_30days,brand_purchased_amt_90days,brand_purchased_qty_90days,brand_purchased_amt_180days,brand_purchased_qty_180days,brand_purchased_amt_270days,brand_purchased_qty_270days,purchased_company_category_brand_count,purchased_company_category_count
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,7.98,2.0,19.95,5.0,19.95,5.0,19.95,5.0,0.0,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,0.00,0.0,3.99,1.0,8.98,2.0,8.98,2.0,0.0,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0


In [120]:
k26.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16000 entries, 0 to 15999
Data columns (total 49 columns):
id                                        16000 non-null int64
total_trans_purchaseamount_avg            16000 non-null float64
12_mnths_total_purchase_amt               16000 non-null float64
9_mnths_total_purchase_amt                16000 non-null float64
6_mnths_total_purchase_amt                16000 non-null float64
3_mnths_total_purchase_amt                16000 non-null float64
1_month_total_purchase_amt                16000 non-null float64
total_purchaseamount                      16000 non-null float64
total_purchasequantity                    16000 non-null int64
total_transactions_count                  16000 non-null int64
offer                                     16000 non-null int64
repeater                                  16000 non-null int64
repeattrips                               16000 non-null int64
offer_company                             16000 non-null int64


<h1 style="color:blue;">Save the dataframe as CSV to use it for model building</h1>

In [121]:
k26.to_csv("../../data/data.csv", sep=',', encoding='utf-8',index_label='index')